In [2]:
import torch
import torch.nn as nn
import pandas as pd

device = torch.device("cuda")


In [13]:
class encoder(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        
        self.encoder = nn.Sequential(
            nn.Linear(input_size, 740),
            nn.ReLU(),
            nn.Linear(740, 640),
            nn.ReLU(),
            nn.Linear(640, 320),
            nn.ReLU(),
            nn.Linear(320, 160),
            nn.ReLU(),
            nn.Linear(160, 80),
            nn.ReLU(),
            nn.Linear(80, 40))
        
        self.decoder = nn.Sequential(
            nn.Linear(40, 80),
            nn.ReLU(),
            nn.Linear(80, 160),
            nn.ReLU(),
            nn.Linear(160, 320),
            nn.ReLU(),
            nn.Linear(320, 640),
            nn.ReLU(),
            nn.Linear(640, 740),
            nn.ReLU(),
            nn.Linear(740, input_size),
            nn.Sigmoid())
    
    def forward(self, x):
        return self.encoder(x)


In [17]:
from torch.utils.data import Dataset

class steamGamesDataset(Dataset):
    def __init__(self, lhs_dataframe, rhs_dataframe, device='cpu'):
        self.lhs_df = lhs_dataframe
        self.rhs_df = rhs_dataframe
        # self.dataset = pd.concat([lhs_dataframe, rhs_dataframe], axis=1)
        
    def __len__(self):
        return len(self.lhs_df)
    
    def __getitem__(self, index):
        inp_features = torch.tensor(self.lhs_df.iloc[index]).to(torch.float32)
        value = torch.tensor(self.rhs_df.iloc[index]).to(torch.float32)
        return inp_features, value


In [4]:
onehot = pd.read_csv('../Data/OneHotVectors.csv')


In [19]:
autoEncoder_dataset = steamGamesDataset(onehot, onehot)
autoEncoder_dataloader = torch.utils.data.DataLoader(autoEncoder_dataset, batch_size=1)


In [26]:
model = encoder(834).to(device)
model.encoder.load_state_dict(torch.load('../Data/encoderparams.pt'))


<All keys matched successfully>

In [32]:
encoded = pd.DataFrame()
for inp, _ in autoEncoder_dataloader:
    inp = inp.to(device)
    output = model.forward(inp)
    encoded = pd.concat([encoded, pd.DataFrame(output.detach().cpu().numpy())])


C:\Users\speed\AppData\Local\Temp\ipykernel_26884\2557768976.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  inp_features = torch.tensor(self.lhs_df.iloc[index]).to(torch.float32)
C:\Users\speed\AppData\Local\Temp\ipykernel_26884\2557768976.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  value = torch.tensor(self.rhs_df.iloc[index]).to(torch.float32)
C:\Users\speed\AppData\Local\Temp\ipykernel_26884\2557768976.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a 

KeyboardInterrupt: 